In [68]:
import sys
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout, LSTM, Flatten
from keras.models import Model
from keras import optimizers,regularizers
from keras.callbacks import History,EarlyStopping
from keras.optimizers import SGD
import keras_metrics
from sklearn import metrics
from sklearn.utils import shuffle
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.datasets import make_classification
from sklearn.utils import shuffle, class_weight
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

In [69]:
labels = pd.read_csv('labels.csv',header=0)
print(list(labels.columns.values))
print("\n")
labels= labels[pd.notnull(labels['gene'])]
# df.value_counts()

    
    
data=pd.read_csv('data.csv',header=0)
print(list(data.columns.values))
data = data[pd.notnull(data['ORF'])]

['gene', 'TCA', 'Resp', 'Ribo', 'Proteas', 'Hist', 'HTH']


['ORF', 'NAME', 'alpha 0', 'alpha 7', 'alpha 14', 'alpha 21', 'alpha 28', 'alpha 35', 'alpha 42', 'alpha 49', 'alpha 56', 'alpha 63', 'alpha 70', 'alpha 77', 'alpha 84', 'alpha 91', 'alpha 98', 'alpha 105', 'alpha 112', 'alpha 119', 'Elu 0', 'Elu 30', 'Elu 60', 'Elu 90', 'Elu 120', 'Elu 150', 'Elu 180', 'Elu 210', 'Elu 240', 'Elu 270', 'Elu 300', 'Elu 330', 'Elu 360', 'Elu 390', 'cdc15 10', 'cdc15 30', 'cdc15 50', 'cdc15 70', 'cdc15 90', 'cdc15 110', 'cdc15 130', 'cdc15 150', 'cdc15 170', 'cdc15 190', 'cdc15 210', 'cdc15 230', 'cdc15 250', 'cdc15 270', 'cdc15 290', 'spo 0', 'spo 2', 'spo 5', 'spo 7', 'spo 9', 'spo 11', 'spo5 2', 'spo5 7', 'spo5 11', 'spo- early', 'spo- mid', 'heat 0', 'heat 10', 'heat 20', 'heat 40', 'heat 80', 'heat 160', 'dtt 15', 'dtt 30', 'dtt 60', 'dtt 120', 'cold 0', 'cold 20', 'cold 40', 'cold 160', 'diau a', 'diau b', 'diau c', 'diau d', 'diau e', 'diau f', 'diau g']


In [70]:
data.head(5)

,ORF,NAME,alpha 0,alpha 7,alpha 14,alpha 21,alpha 28,alpha 35,alpha 42,alpha 49,...,cold 20,cold 40,cold 160,diau a,diau b,diau c,diau d,diau e,diau f,diau g
0,YBR166C,TYR1 TYROSINE BIOSYNTHESIS PREPHENATE DEH...,0.33,-0.17,0.04,-0.07,-0.09,-0.12,-0.03,-0.20,...,0.08,0.39,-0.17,0.23,0.20,0.20,-0.17,-0.69,0.14,-0.27
1,YOR357C,GRD19 SECRETION GOLGI PROTEIN ...,-0.64,-0.38,-0.32,-0.29,-0.22,-0.01,-0.32,-0.27,...,0.36,-0.20,-0.34,-0.14,-0.09,0.06,-0.17,0.04,-0.97,-1.79
2,YLR292C,SEC72 SECRETION ER PROTEIN TRA...,-0.23,0.19,-0.36,0.14,-0.40,0.16,-0.09,-0.12,...,0.21,-0.29,-0.36,-0.10,-0.29,-0.18,-0.34,-0.47,-0.43,-1.06
3,YGL112C,TAF60 TRANSCRIPTION TFIID 60 KD SU...,-0.69,-0.89,-0.74,-0.56,-0.64,-0.18,-0.42,-0.34,...,-0.01,-0.29,0.31,0.21,0.07,0.18,-0.14,-0.20,-0.43,-1.51
4,YIL118W,RHO3 CYTOSKELETON GTP-BINDING PR...,0.04,0.01,-0.81,NaN,-0.30,0.49,0.08,0.19,...,0.28,-0.17,0.26,0.07,-0.17,NaN,-0.10,-0.23,-0.51,-1.40


In [71]:
labels.head(5)

,gene,TCA,Resp,Ribo,Proteas,Hist,HTH
1,YAL001C,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,YAL002W,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,YAL003W,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,YAL005C,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9,YAL010C,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [72]:
X=np.loadtxt('newdata.txt')
Y=np.loadtxt('newlabels.txt')
Y=Y.astype(int)

In [73]:
print(X.shape)
print(Y.shape)
X,Y= shuffle(X,Y)

(2464, 94)
(2464,)


In [74]:
n_values = np.max(Y) + 1
newY=np.eye(n_values)[Y] #one hot encoding for softmax
print(newY.shape)

(2464, 7)


In [75]:
n=int(X.shape[0]*0.80)
length=X.shape[0]
X_train=X[0:n,:]
X_test=X[n:,:]

Y_train=Y[0:n]
Y_test=Y[n:]

newY_train=newY[0:n]
newY_test=newY[n:]

In [76]:
print(X_train.shape)
print(X_test.shape)

(1971, 94)
(493, 94)


In [77]:
print(Y_train.shape)
print(Y_test.shape)

print("One hot encoded Y shape")
print(newY_train.shape)
print(newY_test.shape)

(1971,)
(493,)
One hot encoded Y shape
(1971, 7)
(493, 7)


In [78]:
#using the sklearn class weights to find find the weights of the balanced class
Y_train=Y_train.reshape((Y_train.shape[0],))
print(Y_train.shape)
class_weights_values= class_weight.compute_class_weight('balanced',np.unique(Y_train), Y_train)
Y_train=Y_train.reshape(-1, 1)

(1971,)


In [55]:
seed = 7
np.random.seed(seed)
kfold=KFold(n_splits=4, shuffle=True, random_state=seed)
accuracy=[]
f1=[]
precision=[]
recall=[]

f1_class=np.empty((0,7))
precision_class=np.empty((0,7))
recall_class=np.empty((0,7))
for train, test in kfold.split(X_train, newY_train,Y_train):
    # create model
    model = Sequential()
    model.add(Dense(175, input_dim=94, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(45, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(7, activation='softmax'))
    adam=optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['accuracy'])
    
    model.fit(X_train[train], newY_train[train], epochs=10, batch_size=10, verbose=1,class_weight=class_weights_values)
    y_pred=model.predict(X_train[test])
    predict_class = np.argmax(y_pred, axis=1)
    
    labels=[0,1,2,3,4,5,6]
    #f1 values
    f1_value=metrics.f1_score(predict_class,Y_train[test],labels,average='macro')
    f1_class_value=metrics.f1_score(predict_class,Y_train[test],labels,average=None).reshape(1, -1)
    
    #precision values
    precision_value=metrics.precision_score(predict_class,Y_train[test],labels,average='macro')
    precision_class_value=metrics.precision_score(predict_class,Y_train[test],labels,average=None).reshape(1, -1)
    
    #recall values
    recall_value=metrics.recall_score(predict_class,Y_train[test],labels,average='macro')
    recall_class_value=metrics.recall_score(predict_class,Y_train[test],labels,average=None).reshape(1, -1)
    
    scores = model.evaluate(X_train[test], newY_train[test], verbose=1)
    accuracy.append(scores[1])
    f1.append(f1_value)
    precision.append(precision_value)
    recall.append(recall_value)
    
    f1_class=np.concatenate((f1_class,f1_class_value),axis=0)
    precision_class=np.concatenate((precision_class,precision_class_value),axis=0)
    recall_class=np.concatenate((recall_class,recall_class_value),axis=0)
    
average_f1=f1_class.mean(axis=0)
average_precision=precision_class.mean(axis=0)
average_recall=recall_class.mean(axis=0)

Epoch 1/10
1478/1478 [==============================] - 1s 668us/step - loss: 0.4893 - acc: 0.8884
Epoch 2/10
1478/1478 [==============================] - 0s 135us/step - loss: 0.2443 - acc: 0.9411
Epoch 3/10
1478/1478 [==============================] - 0s 133us/step - loss: 0.2147 - acc: 0.9499
Epoch 4/10
1478/1478 [==============================] - 0s 125us/step - loss: 0.1619 - acc: 0.9648
Epoch 5/10
1478/1478 [==============================] - 0s 129us/step - loss: 0.1517 - acc: 0.9662
Epoch 6/10
1478/1478 [==============================] - 0s 130us/step - loss: 0.1467 - acc: 0.9635
Epoch 7/10
1478/1478 [==============================] - 0s 130us/step - loss: 0.1235 - acc: 0.9668
Epoch 8/10
1478/1478 [==============================] - 0s 129us/step - loss: 0.1338 - acc: 0.9641
Epoch 9/10
1478/1478 [==============================] - 0s 130us/step - loss: 0.1095 - acc: 0.9662
Epoch 10/10
493/493 [==============================] - 0s 592us/step
Epoch 1/10
1478/1478 [==================

In [56]:
print("For training:")
print("Accuracy is :"+ str(np.mean(accuracy)))
print("F1 is :"+ str(np.mean(f1)))
print("Average F1 for all the classes is:" +str(average_f1))

For training:
Accuracy is :0.9690494154229595
F1 is :0.6191780102058136
Average F1 for all the classes is:[0.98306281 0.125      0.82400794 0.95785714 0.59015152 0.85416667
 0.        ]


In [99]:
#Based on the other models, we will compare the score against the other models
#and then see how well the model will generalise based on the test scores.

In [ ]:
seed =7
np.random.seed(seed)
kfold=StratifiedKFold(n_splits=4, shuffle=True,random_state=seed)
accuracy=[]
f1=[]
precision=[]
recall=[]

f1_class=np.empty((0,7))
precision_class=np.empty((0,7))
recall_class=np.empty((0,7))

for train, test in kfold.split(X_train, Y_train):
    # create model
    
    #y_smote=np.argmax(newY_train[train], axis=1)
    
    #automatically balance all the classes of the training data
    smote_tomek = SMOTETomek(random_state=0)
    X_under, y_under=smote_tomek.fit_resample(X_train[train], Y_train[train]) #sampling on the trainng data only
    X_under,y_under= shuffle(X_under,y_under)
    
    
    n_values = np.max(y_under) + 1
    y_under=np.eye(n_values)[y_under] #one hot encoding for softmax
    
    """
    pca= PCA(50)
    X_under= pca.fit_transform(X_under)
    newX_test=pca.transform(X_train[test])
    """
    
    model = Sequential()
    model.add(Dense(175, input_dim=94, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='sigmoid'))
    model.add(Dropout(0.3))
    model.add(Dense(45, activation='sigmoid'))
    model.add(Dropout(0.3))
    model.add(Dense(7, activation='softmax'))
    adam=optimizers.Adam(lr=0.0029, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['accuracy',keras_metrics.precision(), keras_metrics.recall()])
    
    model.fit(X_under, y_under, epochs=10, batch_size=32, verbose=1)
    
    y_pred=model.predict(X_train[test])
    predict_class = np.argmax(y_pred, axis=1) #the predicted value
    
    labels=[0,1,2,3,4,5,6]
    Ylabel= np.argmax(newY_train[test], axis=1) #the actual value
    
    #f1 values
    f1_value=metrics.f1_score(Ylabel,predict_class,labels,average='macro')
    f1_class_value=metrics.f1_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
    
    #precision values
    precision_value=metrics.precision_score(Ylabel,predict_class,labels,average='macro')
    precision_class_value=metrics.precision_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
    
    #recall values
    recall_value=metrics.recall_score(Ylabel,predict_class,labels,average='macro')
    recall_class_value=metrics.recall_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
                                         
    scores = model.evaluate(X_train[test], newY_train[test], verbose=2)
    accuracy.append(scores[1])
    f1.append(f1_value)
    precision.append(precision_value)
    recall.append(recall_value)
    
    f1_class=np.concatenate((f1_class,f1_class_value),axis=0)
    precision_class=np.concatenate((precision_class,precision_class_value),axis=0)
    recall_class=np.concatenate((recall_class,recall_class_value),axis=0)
    
average_f1=f1_class.mean(axis=0)
average_precision=precision_class.mean(axis=0)
average_recall=recall_class.mean(axis=0)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/10
9429/9429 [==============================] - 2s 191us/step - loss: 0.5102 - acc: 0.8509 - precision: 0.9197 - recall: 0.4083
Epoch 2/10
9429/9429 [==============================] - 1s 97us/step - loss: 0.0762 - acc: 0.9847 - precision: 0.9717 - recall: 0.9169
Epoch 3/10
9429/9429 [==============================] - 1s 122us/step - loss: 0.0417 - acc: 0.9916 - precision: 0.9824 - recall: 0.9532
Epoch 4/10
9429/9429 [==============================] - 1s 118us/step - loss: 0.0329 - acc: 0.9920 - precision: 0.9804 - recall: 0.9651
Epoch 5/10
9429/9429 [==============================] - 1s 130us/step - loss: 0.0227 - acc: 0.9948 - precision: 0.9895 - recall: 0.9785
Epoch 6/10
9429/9429 [==============================] - 1s 119us/step - loss: 0.0191 - acc: 0.9958 - precision: 0.9940 - recall: 0.9785
Epoch 7/10
9429/9429 [==============================] - 1s 117us/step - loss: 0.0157 - acc: 0.9960 - precision: 0.9881 - recall: 0.9844
Epoch 8/10
9429/9429 [===========================

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/10
8384/9429 [=========================>....] - ETA: 0s - loss: 0.5359 - acc: 0.8534 - precision: 0.9025 - recall: 0.4182

In [97]:
print("For training:")
print("Accuracy is :"+ str(np.mean(accuracy)))
print("F1 is :"+ str(np.mean(f1)))
print("Average F1 for all the classes is:" +str(average_f1))

For training:
Accuracy is :0.9604129524277589
F1 is :0.650312158083168
Average F1 for all the classes is:[0.97904882 0.29444444 0.62619048 0.9358347  0.8        0.91666667
 0.        ]


In [90]:
seed = 7
np.random.seed(seed)
kfold=StratifiedKFold(n_splits=4, shuffle=True,random_state=seed)
accuracy=[]
f1=[]
precision=[]
recall=[]

f1_class=np.empty((0,7))
precision_class=np.empty((0,7))
recall_class=np.empty((0,7))

for train, test in kfold.split(X_train, Y_train):
    # create model
    
    #y_smote=np.argmax(newY_train[train], axis=1)
    
    #pca for all features
    alphaX= X_train[train][:,0:18]
    eluX= X_train[train][:,18:32]
    edcX= X_train[train][:,32:47]
    spotX= X_train[train][:,47:56]
    spoX= X_train[train][:,56:58]
    heatX= X_train[train][:,58:64]
    dttX= X_train[train][:,64:68]
    coldX= X_train[train][:,68:72]
    diauX= X_train[train][:,72:79]

    alphaX_test= X_train[test][:,0:18]
    eluX_test= X_train[test][:,18:32]
    edcX_test= X_train[test][:,32:47]
    spotX_test= X_train[test][:,47:56]
    spoX_test= X_train[test][:,56:58]
    heatX_test= X_train[test][:,58:64]
    dttX_test= X_train[test][:,64:68]
    coldX_test= X_train[test][:,68:72]
    diauX_test= X_train[test][:,72:79]
    
    pca= PCA(1)
    
    alphaX= pca.fit_transform(alphaX)
    alphaX_test=pca.transform(alphaX_test)
    
    eluX= pca.fit_transform(eluX)
    eluX_test=pca.transform(eluX_test)
    
    edcX= pca.fit_transform(edcX)
    edcX_test=pca.transform(edcX_test)
    
    spotX= pca.fit_transform(spotX)
    spotX_test=pca.transform(spotX_test)
    
    spoX= pca.fit_transform(spoX)
    spoX_test=pca.transform(spoX_test)
    
    heatX= pca.fit_transform(heatX)
    heatX_test=pca.transform(heatX_test)
    
    dttX= pca.fit_transform(dttX)
    dttX_test=pca.transform(dttX_test)
    
    coldX= pca.fit_transform(coldX)
    coldX_test=pca.transform(coldX_test)
    
    diauX= pca.fit_transform(diauX)
    diauX_test=pca.transform(diauX_test)
    
    x_train=np.concatenate([alphaX,eluX,edcX,spotX,spoX,heatX,dttX,coldX,diauX],axis=1)
    x_test=np.concatenate([alphaX_test,eluX_test,edcX_test,spotX_test,spoX_test,heatX_test,dttX_test,coldX_test,diauX_test],axis=1)
    print(x_train.shape)
    print(x_test.shape)
    
    #over and undersampling-balancing all the classes
    smote_tomek = SMOTETomek(random_state=0)
    X_under, y_under=smote_tomek.fit_resample(x_train, Y_train[train])
    X_under,y_under= shuffle(X_under,y_under)
    
    n_values = np.max(y_under) + 1
    y_under=np.eye(n_values)[y_under] #one hot encoding for softmax
    
    """
    pca= PCA(50)
    X_under= pca.fit_transform(X_under)
    newX_test=pca.transform(X_train[test])
    """
    
    model = Sequential()
    model.add(Dense(175, input_dim=9, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(75, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(7, activation='softmax'))
    adam=optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['accuracy',keras_metrics.precision(), keras_metrics.recall()])
    
    model.fit(X_under, y_under, epochs=10, batch_size=32, verbose=1)
    
    y_pred=model.predict(x_test)
    predict_class = np.argmax(y_pred, axis=1) #the predicted value
    
    labels=[0,1,2,3,4,5,6]
    Ylabel= np.argmax(newY_train[test], axis=1) #the actual value
    
    #f1 values
    f1_value=metrics.f1_score(Ylabel,predict_class,labels,average='macro')
    f1_class_value=metrics.f1_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
    
    #precision values
    precision_value=metrics.precision_score(Ylabel,predict_class,labels,average='macro')
    precision_class_value=metrics.precision_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
    
    #recall values
    recall_value=metrics.recall_score(Ylabel,predict_class,labels,average='macro')
    recall_class_value=metrics.recall_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
                                         
    scores = model.evaluate(x_test, newY_train[test], verbose=2)
    accuracy.append(scores[1])
    f1.append(f1_value)
    precision.append(precision_value)
    recall.append(recall_value)
    
    f1_class=np.concatenate((f1_class,f1_class_value),axis=0)
    precision_class=np.concatenate((precision_class,precision_class_value),axis=0)
    recall_class=np.concatenate((recall_class,recall_class_value),axis=0)
    
average_f1=f1_class.mean(axis=0)
average_precision=precision_class.mean(axis=0)
average_recall=recall_class.mean(axis=0)

(1476, 9)
(495, 9)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/10
9429/9429 [==============================] - 1s 157us/step - loss: 0.5916 - acc: 0.7935 - precision: 0.7052 - recall: 0.2806
Epoch 2/10
9429/9429 [==============================] - 1s 82us/step - loss: 0.3202 - acc: 0.8917 - precision: 0.7901 - recall: 0.5561
Epoch 3/10
9429/9429 [==============================] - 1s 73us/step - loss: 0.2464 - acc: 0.9195 - precision: 0.8435 - recall: 0.6481
Epoch 4/10
9429/9429 [==============================] - 1s 88us/step - loss: 0.2097 - acc: 0.9307 - precision: 0.8676 - recall: 0.6956
Epoch 5/10
9429/9429 [==============================] - 1s 75us/step - loss: 0.1703 - acc: 0.9467 - precision: 0.8947 - recall: 0.7506
Epoch 6/10
9429/9429 [==============================] - 1s 82us/step - loss: 0.1520 - acc: 0.9526 - precision: 0.9125 - recall: 0.7743
Epoch 7/10
9429/9429 [==============================] - 1s 74us/step - loss: 0.1358 - acc: 0.9548 - precision: 0.9192 - recall: 0.7854
Epoch 8/10
9429/9429 [==============================] 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/10
9429/9429 [==============================] - 1s 144us/step - loss: 0.5930 - acc: 0.8060 - precision: 0.7482 - recall: 0.3088
Epoch 2/10
9429/9429 [==============================] - 1s 83us/step - loss: 0.3018 - acc: 0.9025 - precision: 0.8284 - recall: 0.5664
Epoch 3/10
9429/9429 [==============================] - 1s 94us/step - loss: 0.2433 - acc: 0.9253 - precision: 0.8663 - recall: 0.6303
Epoch 4/10
9429/9429 [==============================] - 1s 82us/step - loss: 0.1984 - acc: 0.9370 - precision: 0.8889 - recall: 0.6949
Epoch 5/10
9429/9429 [==============================] - 1s 80us/step - loss: 0.1686 - acc: 0.9468 - precision: 0.9009 - recall: 0.7223
Epoch 6/10
9429/9429 [==============================] - 1s 78us/step - loss: 0.1467 - acc: 0.9554 - precision: 0.9181 - recall: 0.7654
Epoch 7/10
9429/9429 [==============================] - 1s 84us/step - loss: 0.1368 - acc: 0.9562 - precision: 0.9211 - recall: 0.7713
Epoch 8/10
9429/9429 [==============================] 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/10
9429/9429 [==============================] - 1s 158us/step - loss: 0.6294 - acc: 0.7844 - precision: 0.7280 - recall: 0.3318
Epoch 2/10
9429/9429 [==============================] - 1s 76us/step - loss: 0.3239 - acc: 0.8924 - precision: 0.8125 - recall: 0.5791
Epoch 3/10
9429/9429 [==============================] - 1s 87us/step - loss: 0.2461 - acc: 0.9163 - precision: 0.8405 - recall: 0.6570
Epoch 4/10
9429/9429 [==============================] - 1s 96us/step - loss: 0.1945 - acc: 0.9382 - precision: 0.8963 - recall: 0.7120
Epoch 5/10
9429/9429 [==============================] - 1s 94us/step - loss: 0.1669 - acc: 0.9450 - precision: 0.9039 - recall: 0.7468
Epoch 6/10
9429/9429 [==============================] - 1s 89us/step - loss: 0.1586 - acc: 0.9475 - precision: 0.9089 - recall: 0.7632
Epoch 7/10
9429/9429 [==============================] - 1s 90us/step - loss: 0.1343 - acc: 0.9562 - precision: 0.9278 - recall: 0.7914
Epoch 8/10
9429/9429 [==============================] 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/10
9427/9427 [==============================] - 2s 163us/step - loss: 0.5840 - acc: 0.8028 - precision: 0.7376 - recall: 0.3195
Epoch 2/10
9427/9427 [==============================] - 1s 85us/step - loss: 0.2887 - acc: 0.9018 - precision: 0.7969 - recall: 0.5713
Epoch 3/10
9427/9427 [==============================] - 1s 78us/step - loss: 0.2144 - acc: 0.9291 - precision: 0.8629 - recall: 0.6642
Epoch 4/10
9427/9427 [==============================] - 1s 80us/step - loss: 0.1776 - acc: 0.9443 - precision: 0.9004 - recall: 0.7184
Epoch 5/10
9427/9427 [==============================] - 1s 84us/step - loss: 0.1531 - acc: 0.9516 - precision: 0.9150 - recall: 0.7437
Epoch 6/10
9427/9427 [==============================] - 1s 83us/step - loss: 0.1372 - acc: 0.9571 - precision: 0.9280 - recall: 0.7660
Epoch 7/10
9427/9427 [==============================] - 1s 82us/step - loss: 0.1251 - acc: 0.9611 - precision: 0.9206 - recall: 0.8016
Epoch 8/10
9427/9427 [==============================] 

In [91]:
print("For training:")
print("Accuracy is :"+ str(np.mean(accuracy)))
print("F1 is :"+ str(np.mean(f1)))
print("Average F1 for all the classes is:" +str(average_f1))

For training:
Accuracy is :0.8498108507029131
F1 is :0.43508917185524265
Average F1 for all the classes is:[0.9150276  0.17045455 0.18411942 0.73566161 0.40583722 0.63452381
 0.        ]
